In [ ]:
install.packages('dplyr')
install.packages('ggplot2')
install.packages('readr')
install.packages('glmnet')
install.packages('Metrics')
install.packages('MLmetrics')
install.packages('MASS')

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [ ]:
require(dplyr)
require(ggplot2)
require(readr)
require(glmnet)
require(Metrics)
require(MLmetrics)
require(MASS)
require(bigrquery)

In [ ]:
# Provide authentication to connect to BigQuery
bq_auth(use_oob = TRUE)

In [ ]:
# Store the GCP project id
projectid = "gcphackathorn"

# Set your query
sql <- "SELECT * FROM dataset_final.final"

# Run query
tb <- bq_project_query(projectid, sql)

# Download query results
df <- bq_table_download(tb)

In [ ]:
head(df)

In [ ]:
names(df)

In [ ]:
Train <- df %>% 
  transmute(killings_hispanic,
            year,
            state,
            poverty_rate,
            poverty_male,
            poverty_hispanic,
            poverty_below_high_school,
            male_perc = round(male/population*100,2),
            hispanic_perc = round(hispanic/population*100,2),
            below_high_school_perc = round(below_high_school/population*100,2),
            unemployment_rate = round(unemployed/population*100,2),
            killings_total
          ) %>%
    mutate(state = as.factor(state)) %>%
         na.omit()   # remove rows with missings

In [ ]:
names(Train)

In [ ]:
head(Train) %>% as.data.frame()

In [ ]:
#Creating objects for regression
y = as.numeric(Train$killings_hispanic)          # our y variable
d = Train$poverty_hispanic                    # our treatment variable

Train =  Train %>% dplyr::select(-killings_hispanic, -poverty_hispanic)    # remove y and d from Train 
X = model.matrix(d ~ ., Train) # select all variables and remove Intercept 

In [ ]:
colnames(X)

# Causal Lasso

## 1st Stage

In [ ]:
install.packages('doParallel')

In [ ]:
# Setting lambda interval
lamb_range = 10 ^ seq(1, -7, length= 100) 

Train2 = as.matrix(Train)
### Runs CV - Lasso

library(doParallel)
registerDoParallel(5)
set.seed(9547)
cv_lasso= cv.glmnet(y = d, 
                    x = X, 
                    alpha = 1, 
                    nfolds = 5,
                    lambda = lamb_range,
                    standardize=TRUE,
                    parallel=TRUE)

In [ ]:
# Plots the MSE errors
plot(cv_lasso)

# Plots the Paths
plot(cv_lasso$glmnet.fit, xvar="lambda")

In [ ]:
# Estimating D Hat

### Retrieves the lambdas
sd1_lambda = cv_lasso$lambda.1se

# Mean squared errors from best lambdas
print(cv_lasso$cvm[which(cv_lasso$lambda == cv_lasso$lambda.1se)])

# In-sample dhats
dhat_1se = predict(object = cv_lasso, s = "lambda.1se", newx = X)
colnames(dhat_1se) <- "dhat"

# Plot d against dhat
plot(dhat_1se, d)

## 2nd Stage

In [ ]:
X2 = cbind(dhat=dhat_1se, treatment = d, X)

In [ ]:
# Leave dhat unpenalized
penalty_list = rep(1,ncol(X2))
penalty_list[1] = 0

In [ ]:
### Run Lasso with dhat unpenalized
set.seed(17)

cv_lasso_final = cv.glmnet(y = y, 
                    x = X2, 
                    alpha = 1, 
                    nfolds = 5,
                    penalty.factor = penalty_list,
                    lambda = lamb_range,
                    standardize=TRUE,
                    parallel=TRUE)

# Plots the MSE errors
plot(cv_lasso_final)

# Plots the Paths
plot(cv_lasso_final$glmnet.fit, xvar="lambda")

In [ ]:
cv_lasso_final

In [ ]:
# Number of variables selected by Lasso
print("Total variables left:")
print(sum(coef(cv_lasso_final, select="1se") != 0))    # 43 variables survived

# Optimal lambda: 
lambda_final = cv_lasso_final$lambda.1se # 0.0053
print("final lambda:")
print(lambda_final)

#### !!! Coefficient of Degree variable after controlling for unpenalized dhat. 
impact = coef(cv_lasso_final, select = "lambda.min")["treatment",]
print('impact:')
impact

In [ ]:
# Mean The mean cross-validated error from best lambda
cv_lasso_final$cvm[which(cv_lasso_final$lambda == cv_lasso_final$lambda.1se)]

### AIC
fit2 <- glmnet(y = y,
               x = X2,
               alpha = 1,
               penalty.factor = penalty_list,
               lambda = cv_lasso_final$lambda.1se,
               standardize=TRUE)

tLL2 <- fit2$nulldev - deviance(fit2)
k2 <- fit2$df
n2 <- fit2$nobs
AICc2 <- -tLL2+2*k2+2*k2*(k2+1)/(n2-k2-1)
AICc2

fit2$dev.ratio  # considered R squared

## Bootstrap

In [ ]:
n <- nrow(X2)
betas <- c()

for(ind in 1:100){
  print(ind)
  sam_boot <- sample(1:n, n, replace=T)
  lfit = glmnet(y = y[sam_boot], 
                x = X2[sam_boot,], 
                alpha = 1, 
                penalty.factor = penalty_list,
                lambda = lambda_final,  # min lambda from 3rd question
                standardize=TRUE)
  degree_coef = coef(lfit)["treatment",]
  betas[ind] <- as.numeric(degree_coef)
  
}

betas <- sort(betas)

print(quantile(betas, c(.025, 0.975)))  # confidence interval

# Print histogram of betas (booststrap)
hist(betas, nclass = 20)
abline(v=impact, col=2)

boxplot(betas)